<a href="https://colab.research.google.com/github/alex0249/sankey/blob/main/energy_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy consumption

This notebook is split into 3 tasks:


1.   US Energy Sankey example
2.   Create your own Sankey using IEA data
3.   Automate Sankey generation for any country by writing a Python function

Run the first two cells setup the notebook.





In [13]:
# Installation and downloads
!pip install floweaver ipysankeywidget openpyxl --upgrade

from floweaver import *
import gdown, os, zipfile

def download_and_unzip_from_google_drive(file_id='1qriY29v7eKJIs07UxAw5RlJirfwuLnyP', destination_folder='example_data'):
    if os.path.exists(destination_folder):
        print('Zip file already downloaded. If you want to reload, please delete the example_data folder.')
        return
    else:
        download_url = f"https://drive.google.com/uc?id={file_id}"
        output = 'temp.zip'
        print("Downloading file...")
        gdown.download(download_url, output, quiet=False)
        os.makedirs(destination_folder, exist_ok=True)
        print("Extracting files...")
        with zipfile.ZipFile(output, 'r') as zip_ref:
            zip_ref.extractall(destination_folder)
        os.remove(output)
        print("Download and extraction complete.")

download_and_unzip_from_google_drive()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.1 MB/s eta 0:00:00


Downloading...
From: https://drive.google.com/uc?id=1qriY29v7eKJIs07UxAw5RlJirfwuLnyP
To: /content/temp.zip
100%|██████████| 154k/154k [00:00<00:00, 53.4MB/s]

Extracting files...
Download and extraction complete.


In [2]:
"""Display setup"""
# Enable widget display for Sankeys in Colab
from google.colab import output
output.enable_custom_widget_manager()

## Task 1 - US example

Step through this section to see an example for the US based on the [Sankey diagrams of US energy consumption from the Lawrence Livermore National Laboratory](https://flowcharts.llnl.gov/).

In [4]:
"""Load the dataset"""
dataset = Dataset.from_csv('example_data/us-energy-consumption.csv',
                           dim_process_filename='example_data/us-energy-consumption-processes.csv')

In [5]:
"""Define the order the nodes appear in"""
sources = ['Solar', 'Nuclear', 'Hydro', 'Wind', 'Geothermal',
           'Natural_Gas', 'Coal', 'Biomass', 'Petroleum']

uses = ['Residential', 'Commercial', 'Industrial', 'Transportation']

In [6]:
"""define the Sankey diagram definition"""
nodes = {
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    'imports': ProcessGroup(['Net_Electricity_Import'], title='Net electricity imports'),
    'electricity': ProcessGroup(['Electricity_Generation'], title='Electricity Generation'),
    'uses': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses)),

    'energy_services': ProcessGroup(['Energy_Services'], title='Energy services'),
    'rejected': ProcessGroup(['Rejected_Energy'], title='Rejected energy'),

    'direct_use': Waypoint(Partition.Simple('source', [
        # This is a hack to hide the labels of the partition, there should be a better way...
        (' '*i, [k]) for i, k in enumerate(sources)
    ])),
}

ordering = [
    [[], ['sources'], []],
    [['imports'], ['electricity', 'direct_use'], []],
    [[], ['uses'], []],
    [[], ['rejected', 'energy_services'], []]
]

bundles = [
    Bundle('sources', 'electricity'),
    Bundle('sources', 'uses', waypoints=['direct_use']),
    Bundle('electricity', 'uses'),
    Bundle('imports', 'uses'),
    Bundle('uses', 'energy_services'),
    Bundle('uses', 'rejected'),
    Bundle('electricity', 'rejected'),
]

In [7]:
"""Define the colours to roughly imitate the original Sankey diagram"""
palette = {
    'Solar': 'gold',
    'Nuclear': 'red',
    'Hydro': 'blue',
    'Wind': 'purple',
    'Geothermal': 'brown',
    'Natural_Gas': 'steelblue',
    'Coal': 'black',
    'Biomass': 'lightgreen',
    'Petroleum': 'green',
    'Electricity': 'orange',
    'Rejected energy': 'lightgrey',
    'Energy services': 'dimgrey',
}

And here's the result!

In [8]:
sdd = SankeyDefinition(nodes, bundles, ordering,
                       flow_partition=dataset.partition('type'))
weave(sdd, dataset, palette=palette) \
    .to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

You can save a copy of the Sankey by adding `.auto_save_png('filename.png')` or `.auto_save_svg('filename.svg')` to the end of the `weave` call in the previous box.

## Task 2 - Create your own

Follow the steps below to create an equivalent Sankey for a different country.

In [14]:
# @markdown   1. Download the IEA World Energy Balances spreadsheet, by running this cell. Further information is available at https://www.iea.org/data-and-statistics/data-product/world-energy-statistics-and-balances. Open the file up on your computer and navigate to the `TimeSeries_1971-2022` sheet. Save this sheet as a .csv file which is much easier to read into most programmes than an Excel file. Then upload it to Colab using the `upload` button in the left panel or by simply dragging and dropping.
%%capture
file_id = '1FN-0xkVHiM2V75RTgl-Aq7l862mgWdAh'
download_url = f"https://drive.google.com/uc?id={file_id}"
!wget -O WorldEnergyBalances.xlsx $download_url
files.download('WorldEnergyBalances.xlsx')

  2. In the next cell import the sheet to a pandas DataFrame. To find appropriate functions for the next steps either have a look at the [pandas documentation](https://pandas.pydata.org/docs/reference/index.html), or remember [your best friend](https://www.google.com/) when writing code.



In [2]:
"""Read in an Excel file"""
import pandas as pd
from floweaver import *
fileName = 'WorldEnergyBalances.csv'
data = pd.read_csv(fileName)

3. Filter the DataFrame to contain only the desired country data.

In [5]:
"""Get desired country"""
country = "Korea"
countryData = data.loc[data['Country'] == country]

4. Filter the DataFrame to only contain 'Product', 'Flow' and value for the latest full year. To get the latest year, find the maximum integer value in the column headers.

In [6]:
"""Get values for latest year"""
lastYear = '2022'
filterData = countryData[['Product', 'Flow', lastYear]]

# Display data
display(filterData)

,Product,Flow,2022
2160,"Coal, peat and oil shale",Production (PJ),15
2161,"Coal, peat and oil shale",Imports (PJ),2855
2162,"Coal, peat and oil shale",Exports (PJ),0
2163,"Coal, peat and oil shale",Total energy supply (PJ),2856
2164,"Coal, peat and oil shale","Electricity, CHP and heat plants (PJ)",-1931
...,...,...,...
2267,Total,Other final consumption (PJ),2324
6668,Fossil fuels,Electricity output (GWh),383850
6669,Nuclear,Electricity output (GWh),176054
6670,Renewable sources,Electricity output (GWh),45400


5. Filter out rows containing summaries (i.e. Total, Production), different units (GWh) or non-numeric values.

In [7]:
"""Filter out Totals and bad characters"""
# Remove rows
remove = '|'.join(['Production','Total','GWh'])
filterData = filterData[~filterData['Product'].str.contains(remove, na=False)]
filterData = filterData[~filterData['Flow'].str.contains(remove, na=False)]

# Convert input values to numeric

filterData[lastYear] = pd.to_numeric(filterData[lastYear], errors='coerce')
filterData = filterData.dropna(subset=[lastYear])

6. Let's match the format in the files for the US example that you can find in the 'example_data' folder.

In [9]:
"""Create dataset table"""
import numpy as np
# Rename the columns to define source, target and value
filterData.rename(columns={'Flow': 'source', 'Product': 'target', lastYear: 'value'}, inplace=True)

# Create a dictionary that maps specific targets to your desired 'type' labels
type_map = {
    'Renewables and waste': 'Renewables and waste',
    'Coal, peat and oil shale': 'Coal, peat and oil shale',
    'Crude, NGL and feedstocks': 'Crude, NGL and feedstocks',
    'Oil products': 'Oil products',
    'Natural gas': 'Natural gas',
    'Nuclear': 'Nuclear',
    'Electricity': 'Electricity',
    'Heat': 'Heat',
    'Total': 'Total'
}

# Assign 'type' based on the 'target' value (default to 'Other' if no match)
filterData['type'] = filterData['target'].map(type_map).fillna('Other')

# Get absolute values to display exports
filterData['value'] = filterData['value'].abs()

# Create groupings - attributes all rows with the right element to a group defined by the left element
groups = [['Electricity','Electricity'], ['Oil products','Oil refineries']]
for g in groups:
    filterData['target'] = [
        g[0] if g[1] in row['target'] else row['target']
        for _, row in filterData.iterrows()
    ]

# Order data so that imports are considered a source and not a target
orderData = filterData.copy()
importRows = np.where(['Imports' in t for t in filterData['target']])[0]

# Swap source and target for those rows
orderData.loc[importRows, 'source'] = filterData.loc[importRows, 'target']
orderData.loc[importRows, 'target'] = filterData.loc[importRows, 'source']

display(orderData)


,target,source,value,type
2161,"Coal, peat and oil shale",Imports (PJ),2855,"Coal, peat and oil shale"
2162,"Coal, peat and oil shale",Exports (PJ),0,"Coal, peat and oil shale"
2164,"Coal, peat and oil shale","Electricity, CHP and heat plants (PJ)",1931,"Coal, peat and oil shale"
2165,"Coal, peat and oil shale","Oil refineries, transformation (PJ)",0,"Coal, peat and oil shale"
2167,"Coal, peat and oil shale",Industry (PJ),340,"Coal, peat and oil shale"
...,...,...,...,...
2251,Heat,Industry (PJ),140,Heat
2252,Heat,Transport (PJ),0,Heat
2253,Heat,Residential (PJ),87,Heat
2254,Heat,Commercial and public services (PJ),13,Heat


7. Let's display all the individual sources and targets and attribute them to process groups for our Sankey diagram.

In [10]:
"""Display individual sources and targets"""
display(orderData['source'].unique(), orderData['target'].unique())

array(['Imports (PJ)', 'Exports (PJ)',
       'Electricity, CHP and heat plants (PJ)',
       'Oil refineries, transformation (PJ)', 'Industry (PJ)',
       'Transport (PJ)', 'Residential (PJ)',
       'Commercial and public services (PJ)',
       'Other final consumption (PJ)'], dtype=object)

array(['Coal, peat and oil shale', 'Crude, NGL and feedstocks',
       'Oil products', 'Natural gas', 'Nuclear', 'Renewables and waste',
       'Electricity', 'Heat'], dtype=object)

In [11]:
"""Attribute to process groups"""

# Sources: Primary energy resources
sources = [
    'Coal, peat and oil shale',
    'Crude, NGL and feedstocks',
    'Oil products',
    'Natural gas',
    'Nuclear',
    'Renewables and waste'
]

# Electricity (and heat) as a separate group (optional):
electricity = [
    'Electricity',
    'Heat'
]

# Imports
imports = [
    'Imports (PJ)'
]

# Exports
exports = [
    'Exports (PJ)'
]

# Refining or transformation processes
refining = [
    'Oil refineries, transformation (PJ)'
]

# Uses: end-use sectors
uses = [
    'Industry (PJ)',
    'Transport (PJ)',
    'Residential (PJ)',
    'Commercial and public services (PJ)',
    'Other final consumption (PJ)'
]


8. Create process table as in us-energy-consumption-processes.csv

In [12]:
"""Get all unique types of sources and targets listed in products and flows respectively"""
idColumn = np.concatenate((sources,uses))
typeColumn = ['source']*len(sources)+['use']*len(uses)
processes = pd.DataFrame(np.array([idColumn,typeColumn]).transpose(), columns=['id','type'])

We now have the same tables as used in the US example. Time to build our own Sankey!

In [14]:
"""Load the dataset"""
dataset = Dataset(orderData, dim_process=processes.set_index('id'))

9. Fetch the Sankey definition for the US energy consumption example from the `"""define the Sankey diagram definition"""` box, adapt this to fit with your new process groups defined in step 7. Adapting the flows can be quite fiddly, so start simple by commenting most lines out and begin with just 2 nodes, their ordering and a bundle linking them. Build this up by thinking effectively about the order of the flows. If you've spent over 30 minutes trying to work this out without success consider the solution sheet, but it's worth getting a bit frustrated first to understand how these flows hold together.

In [ ]:
"""Define the Sankey diagram definition"""
xnnodes = {
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    'imports': ProcessGroup(['Net_Electricity_Import'], title='Net electricity imports'),
    'electricity': ProcessGroup(['Electricity_Generation'], title='Electricity Generation'),
    'uses': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses)),

    'energy_services': ProcessGroup(['Energy_Services'], title='Energy services'),
    'rejected': ProcessGroup(['Rejected_Energy'], title='Rejected energy'),

    'direct_use': Waypoint(Partition.Simple('source', [
        # This is a hack to hide the labels of the partition, there should be a better way...
        (' '*i, [k]) for i, k in enumerate(sources)
    ])),
}

ordering = [
    [[], ['sources'], []],
    [['imports'], ['electricity', 'direct_use'], []],
    [[], ['uses'], []],
    [[], ['rejected', 'energy_services'], []]
]

bundles = [
    Bundle('sources', 'electricity'),
    Bundle('sources', 'uses', waypoints=['direct_use']),
    Bundle('electricity', 'uses'),
    Bundle('imports', 'uses'),
    Bundle('uses', 'energy_services'),
    Bundle('uses', 'rejected'),
    Bundle('electricity', 'rejected'),
]

In [ ]:
"""Define the colours to roughly imitate the original Sankey diagram"""
palette = {
    ##Complete here##
}

In [ ]:
"""Draw out Sankey"""
sdd = SankeyDefinition(nodes, bundles, ordering,
                       flow_partition=dataset.partition('type'))
weave(sdd, dataset, palette=palette) \
    .to_widget(width=900, height=500, margins=dict(left=100, right=200)) \
.auto_save_svg(country+'Sankey.svg')

# Task 3 - Let's automate this procedure for any country with just one click.

Define a function that incorporates all of the previous steps while thinking about still being able to modify it from the outside. The `%%writefile` command writes this function to a Python file when the cell is run.

In [ ]:
%%writefile draw_sankey.py
import numpy as np
import pandas as pd
from floweaver import *

def draw_Country_Sankey()##Complete here##:
    """This function creates a Sankey diagram for the data contained in the specified
    .xlsx file according to the specified parameters"""
      ##Complete here##

    # Function definitions for each processing stage
    def get_country_data()##Complete here##:
        """Extract data for desired country from Excel sheet to pandas dataFrame"""
        ##Complete here##

    def filter_data()##Complete here##:
        """Filter dataFrame according to year and removing rows with unused data"""
        ##Complete here##

    def format_data()##Complete here##:
        """Format dataFrame to be in correct format for floweaver"""
        ##Complete here##

    def group_processes():
        """Group inputs and outputs that represent same process"""
        ##Complete here##

    def reorder_data()##Complete here##:
        """Order data so that imports are considered a source and not a target"""
        ##Complete here##

    def create_process_df()##Complete here##:
        """Get all unique types of sources and targets listed in products and flows respectively"""
        ##Complete here##

    # Define default parameters
    params={
        ##Complete here##
    }

    diagramParams={
        ##Complete here##
    }

    # Update default parameters if other parameters are passed to the function
    ##Complete here##

    # Data processing
    countryData = get_country_data(##Complete here##)
    filterData, year = filter_data(##Complete here##)
    formattedData = format_data(##Complete here##)
    groupData = group_processes(##Complete here##)
    orderedData = reorder_data(##Complete here##)

    # Create processes
    processes = create_process_df(##Complete here##)

    # Create Sankey Dataset
    dataset = Dataset(##Complete here##)

    # Return sankey diagram
    sdd = SankeyDefinition(##Complete here##, flow_partition=dataset.partition('type'))
    return weave(##Complete here##).to_widget(width=900, height=500, margins=dict(left=100, right=200)) \
    .auto_save_svg(##Complete here##)


Now import the function you've created from the written file. This could be done from any Jupyter notebook or python script.

In [ ]:
# Import function
import draw_sankey as ds

# Reloads function in case you've already imported it to this notebook
from importlib import reload
reload(ds)

In [ ]:
# Define variables
fileName = 'WorldEnergyBalances.csv'
country = 'France'
year = '2021'

# Call function
diagram = ds.draw_Country_Sankey(fileName, {'year':year, 'countryName':country})

# Display Sankey
display(diagram)

If this has worked then have a play around and make sure it works for any country by simply modifying the `country` variable.

The solution example is written according to the principles of functional programming, but this could also be written according to object-oriented programming by defining the sankey diagram as a `Class`. If you are a Python expert already and feeling bored why not have a go at re-writing the draw_sankey.py file as a Class definition. Alternatively you could solidify the function you've created with error messages to catch bad inputs or even create a widget which allows you to pick the country you're after off a list rather than typing in the variable.